# Packages

In [26]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import time

from constants import Constant as C
from loaders import load_ratings
from models import ContentBased

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# How to generate predictions for the hackathon ?
To submit predictions to the hackathon: 
- be sure to consider your latest version of ContentBased. Use your evaluator notebook to assess the quality of your model
- run the make_hackathon_prediction() function in the present notebook with your best feature_method and regressor_method. This will generate a ratings_prediction.csv file
- download ratings_prediction.csv and upload it on the onedrive of your group

In [27]:
def make_hackathon_prediction(feature_method=None, regressor_method=None, model=None):
    """Generate a prediction file on the test set"""
    # 1) load train data
    assert str(C.DATA_PATH) == r"C:\Users\nicol\Documents\GitHub\Majeur-BA\RECOMMENDER-SYSTEM\mlsmm2156\data\hackathon"
    sp_ratings = load_ratings(surprise_format=True)
    train_set = sp_ratings.build_full_trainset()
    
    
    content_knn = ContentBased(feature_method, regressor_method)
    content_knn = model
    
    content_knn.fit(train_set)
    
    # 3) make predictions on the test set
    df_test = pd.read_csv('data/hackathon/evidence/ratings_test.csv')[C.USER_ITEM_RATINGS]
    test_records = list(df_test.to_records(index=False))
    predictions = content_knn.test(test_records)
    output_predictions = []
    for uid, iid, _, est, _ in predictions:
        output_predictions.append([uid, iid, est])
    df_predictions = pd.DataFrame(data=output_predictions, columns=df_test.columns)

    # 4) dump predictions
    filename = time.strftime("%Y_%m_%d_%H_%M_%S") + feature_method + "_ratings_predictions.csv"
    filepath = C.RATINGS_PATH / filename
    df_predictions.to_csv(filepath, index=False)
    print(f"Predictions saved to {filepath}")

    
print("\n--- Test avec feature selection ---")
feature_methods = [
    "all_features_with_genome_and_visuals",
    "genome_tags",
    "visual_features"
]

regressor_methods = [
    "neural_network",
    "xgboost",    
    "random_forest"
]

# Configurations du réseau de neurones
default_nn_configs = [
    {
        'name': 'config_1',
        'hidden_layers': [256, 128, 64],
        'dropout_rates': [0.3, 0.2, 0.1],
        'activation': 'relu'
    },
]

for feature_method in feature_methods:
    for regressor in regressor_methods:
        print(f"\n--- Test avec {feature_method} et {regressor} (avec feature selection) ---")
        # Ajouter la configuration nn_config si c'est un réseau de neurones
        if regressor == "neural_network":
        
            feature_method=feature_method,
            regressor_method=regressor,
            nn_config=default_nn_configs
            make_hackathon_prediction(feature_method, regressor, nn_config)
            
        else:
            make_hackathon_prediction(feature_method, regressor)


--- Test avec feature selection ---

--- Test avec all_features_with_genome_and_visuals et neural_network (avec feature selection) ---


NameError: name 'torch' is not defined